In [151]:
#Import libraries

import requests
from requests import get

from bs4 import BeautifulSoup
from os.path  import basename

import pandas as pd
import numpy as np

from time import sleep
from random import randint

In [152]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

In [153]:
#Process League Table
page = 'https://www.transfermarkt.co.uk/laliga/startseite/wettbewerb/ES1/plus/?saison_id=2021'

tree = requests.get(page, headers = headers)
soup = BeautifulSoup(tree.content, 'html.parser')

#Create an empty list to assign these values to
teamLinks = []

#Extract all links with the correct CSS selector
links = soup.select("a.vereinprofil_tooltip")

#We need the location that the link is pointing to, so for each link, take the link location. 
#Additionally, we only need the links in locations 1,3,5,etc. of our list, so loop through those only
for i in range(1,61,3):
    teamLinks.append(links[i].get("href"))
    
#For each location that we have taken, add the website before it - this allows us to call it later
for i in range(len(teamLinks)):
    teamLinks[i] = "https://www.transfermarkt.co.uk"+teamLinks[i]

In [154]:
teamLinks

['https://www.transfermarkt.co.uk/real-madrid/startseite/verein/418/saison_id/2021',
 'https://www.transfermarkt.co.uk/atletico-madrid/startseite/verein/13/saison_id/2021',
 'https://www.transfermarkt.co.uk/fc-barcelona/startseite/verein/131/saison_id/2021',
 'https://www.transfermarkt.co.uk/fc-sevilla/startseite/verein/368/saison_id/2021',
 'https://www.transfermarkt.co.uk/real-sociedad-san-sebastian/startseite/verein/681/saison_id/2021',
 'https://www.transfermarkt.co.uk/fc-villarreal/startseite/verein/1050/saison_id/2021',
 'https://www.transfermarkt.co.uk/fc-valencia/startseite/verein/1049/saison_id/2021',
 'https://www.transfermarkt.co.uk/real-betis-sevilla/startseite/verein/150/saison_id/2021',
 'https://www.transfermarkt.co.uk/athletic-bilbao/startseite/verein/621/saison_id/2021',
 'https://www.transfermarkt.co.uk/fc-getafe/startseite/verein/3709/saison_id/2021',
 'https://www.transfermarkt.co.uk/celta-vigo/startseite/verein/940/saison_id/2021',
 'https://www.transfermarkt.co.uk

In [155]:
#Create an empty list for our player links to go into
playerLinks = []

#Run the scraper through each of our 20 team links
for i in range(len(teamLinks)):

    #Download and process the team page
    page = teamLinks[i]
    tree = requests.get(page, headers = headers)
    soup = BeautifulSoup(tree.content, 'html.parser')

    #Extract all links
    links = soup.select("a.spielprofil_tooltip")
    
    #For each link, extract the location that it is pointing to
    for j in range(len(links)):
        playerLinks.append(links[j].get("href"))

#Add the location to the end of the transfermarkt domain to make it ready to scrape
for j in range(len(playerLinks)):
    playerLinks[j] = "https://www.transfermarkt.co.uk"+playerLinks[j]

#The page list the players more than once - let's use list(set(XXX)) to remove the duplicates
playerLinks = list(set(playerLinks))
playerLinks

['https://www.transfermarkt.co.uk/ivan-chapela/profil/spieler/412046',
 'https://www.transfermarkt.co.uk/david-alaba/profil/spieler/59016',
 'https://www.transfermarkt.co.uk/alvaro-jimenez/profil/spieler/282406',
 'https://www.transfermarkt.co.uk/nestor-araujo/profil/spieler/64134',
 'https://www.transfermarkt.co.uk/lucas-perez/profil/spieler/73185',
 'https://www.transfermarkt.co.uk/unai-garcia/profil/spieler/197122',
 'https://www.transfermarkt.co.uk/dani-vivian/profil/spieler/524000',
 'https://www.transfermarkt.co.uk/damian-suarez/profil/spieler/76746',
 'https://www.transfermarkt.co.uk/jorge-de-frutos/profil/spieler/455181',
 'https://www.transfermarkt.co.uk/john-guidetti/profil/spieler/98596',
 'https://www.transfermarkt.co.uk/iker-muniain/profil/spieler/54235',
 'https://www.transfermarkt.co.uk/david-soria/profil/spieler/226012',
 'https://www.transfermarkt.co.uk/pau-torres/profil/spieler/399776',
 'https://www.transfermarkt.co.uk/stole-dimitrievski/profil/spieler/165467',
 'htt

In [156]:
#Create the df
df = pd.DataFrame()
#Run the scraper through each of the players
for i in range(len(playerLinks)):    
#Download and process the players page
    page = playerLinks[i]
    tree = requests.get(page, headers=headers)
    soup = BeautifulSoup(tree.content, 'html.parser')
#Extract info
    nombre_caracteristicas= soup.find_all("span", { "class" : "dataItem"})
    valor_caracteristicas= soup.find_all("span", {"class" : "dataValue"})
    try:
        market_value=soup.find_all("div", {"class" : "right-td"})
    except:
        market_value = "NA"
    info=playerLinks[i].split('/')
    nombre_apellido=info[3]
    fecha_nacimiento_edad = valor_caracteristicas[0].text.rstrip('</span>').strip().replace(' ','').split('(')
    fecha_nacimiento = fecha_nacimiento_edad[0]
    edad = fecha_nacimiento_edad[1].replace(')','')
    lugar_nacimiento = valor_caracteristicas[1] .text.rstrip('</span>').strip().replace(' ','')
    nacionalidad = valor_caracteristicas[2].text.rstrip('</span>').strip().replace(' ','')
    altura = valor_caracteristicas[3].text.rstrip('</span>').strip().replace(' ','')
    position = valor_caracteristicas[4].text.rstrip('</span>').strip().replace(' ','')
    contract = valor_caracteristicas[5].text.rstrip('</span>').strip().replace(' ','')
    try:
        current_market_value=market_value[0].text.rstrip('</div>').strip().replace(' ','')
    except:
        current_market_value = "NA"

    dfi= pd.DataFrame ({'Link':[playerLinks[i]],'Nom':[nombre_apellido],'Data naixement':[fecha_nacimiento],'Edat':[edad],
                        'Lloc naixement':[lugar_nacimiento],'Nacionalitat':[nacionalidad],
                        'Alçada':[altura], 'Posició' : [position], 'Contracte' : [contract], 'Valor mercat actual': [current_market_value]})    
    df=df.append(dfi)

In [157]:
#Final dataset
df

,Link,Nom,Data naixement,Edat,Lloc naixement,Nacionalitat,Alçada,Posició,Contracte,Valor mercat actual
0,https://www.transfermarkt.co.uk/ivan-chapela/p...,ivan-chapela,"May21,1999",22,Chiclanadela...,Spain,LeftWinger,"Jun30,2022",SpainU17,£225Th.
0,https://www.transfermarkt.co.uk/david-alaba/pr...,david-alaba,"Jun24,1992",29,Wien,Austria,"1,80m",Centre-Back,"Jun30,2026",£49.50m
0,https://www.transfermarkt.co.uk/alvaro-jimenez...,alvaro-jimenez,"May19,1995",26,Córdoba,Spain,"1,76m",RightWinger,"Jun30,2025",£540Th.
0,https://www.transfermarkt.co.uk/nestor-araujo/...,nestor-araujo,"Aug29,1991",30,Guadalajara,Mexico,"1,88m",Centre-Back,"Jun30,2023",£6.30m
0,https://www.transfermarkt.co.uk/lucas-perez/pr...,lucas-perez,"Sep10,1988",33,ACoruña,Spain,"1,80m",Centre-Forward,"Jun30,2022",£2.25m
...,...,...,...,...,...,...,...,...,...,...
0,https://www.transfermarkt.co.uk/renan-lodi/pro...,renan-lodi,"Apr8,1998",23,Serrana(São...,Brazil,"1,78m",Left-Back,"Jun30,2025",£22.50m
0,https://www.transfermarkt.co.uk/hector-belleri...,hector-bellerin,"Mar19,1995",26,Badalona,Spain,"1,78m",Right-Back,"Jun30,2022",£18.00m
0,https://www.transfermarkt.co.uk/aihen-munoz/pr...,aihen-munoz,"Aug16,1997",24,Etxauri,Spain,"1,75m",Left-Back,"Jun30,2022",£5.40m
0,https://www.transfermarkt.co.uk/kike-garcia/pr...,kike-garcia,"Nov25,1989",31,MotilladePalancar,Spain,"1,86m",Centre-Forward,"Jun30,2024",£2.52m


In [158]:
#To csv with the correct encoding
df.to_csv(r'\Users\jordi\Desktop\Master Data Science\2 - Tipologia i cicle de vida de les dades\M2 - Web scraping\dataset.csv',index = False, encoding='utf-8-sig')